# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
# Output File (CSV) from Weatherpy
output_data_file = "../output_data/cities.csv"

#Import csv file from the weatherpy homework
cities = pd.read_csv(output_data_file)

#Display dataframe
cities


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Thompson,55.7435,-97.8558,-3.00,54,90,7.20,CA,1618873983
1,Saint George,37.1041,-113.5841,27.22,7,1,7.72,US,1618873857
2,Puerto Ayora,-0.7393,-90.3518,28.00,74,40,5.14,EC,1618873984
3,Hithadhoo,-0.6000,73.0833,28.07,65,86,2.16,MV,1618873984
4,Longyearbyen,78.2186,15.6401,0.00,93,75,5.66,SJ,1618873985
...,...,...,...,...,...,...,...,...,...
550,Haikou,20.0458,110.3417,25.00,94,75,2.00,CN,1618874272
551,Rancho Palos Verdes,33.7445,-118.3870,23.89,53,1,7.72,US,1618874028
552,Beloha,-25.1667,45.0500,19.79,76,0,5.70,MG,1618874273
553,Omsukchan,62.5333,155.8000,-0.51,89,100,3.13,RU,1618874273


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#Latitude and longitude as heat map
locations = cities[["Latitude", "Longitude"]]
humidity = cities["Humidity"].astype(float)

In [49]:
#Set up a figure variable
fig = gmaps.figure()

#heat layer variable and specifications
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

#add heatlayer to fig
fig.add_layer(heat_layer)

#Display fig
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
#Use .loc to narrow cities with criterias listed inside of the .loc
myweather = cities.loc[(cities["Humidity"]<60)&
                         (cities["Humidity"]<40)&
                         (cities["Cloudiness"]==0)&
                         (cities["Max Temp"]>21)&
                         (cities["Max Temp"]<26)]

#drop the NaN
myweather = myweather.dropna()

#Display the filtered data via weather conditions
myweather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
284,Riyadh,24.6877,46.7219,25.00,19,0,2.24,SA,1618873556
448,Bānsi,27.1833,82.9333,25.18,16,0,1.97,IN,1618874217
459,Tārūt,26.5733,50.0369,23.00,27,0,5.14,SA,1618874222


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
#Url to use for search nearby from the google maps api
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set the hotel dataframe for the final repository of data for the search results/request
hotel_df = pd.DataFrame(columns=["Hotel Name","Latitude", "Longitude", "Address","City", "Country"])

#Use iterrows to expedite and cal out specific city/rows of data
for i, city in pickweather.iterrows():
    
#Set parameters for the search according the google api website criteria
    params={"location": f'{city["Latitude"]},{city["Longitude"]}',
            "keyword": "hotel",
            "radius": 5000,
            "key": g_key}

#Set a request/response visual output for successful and unsucessful requests    
    try:
        response = requests.get(url, params=params)
        response_json=response.json()
        hotel_df=hotel_df.append({"Hotel Name":response_json["results"][0]["name"],
                         "Latitude":city['Latitude'],
                         "Longitude": city['Longitude'],
                         "Address": response_json["results"][0]["vicinity"],
                                  "Country": city["Country"], 
                                  "City": city["City"]},ignore_index=True)
        print(f'{city["City"]} hotel found!')
    except:
        print(f'{city["City"]} does not have a hotel within proximity...')

Bend hotel found!
Adrar does not have a hotel within proximity...
Riyadh hotel found!
Sakakah hotel found!
Mandalī does not have a hotel within proximity...
Bikaner hotel found!
Aswān hotel found!


In [52]:
#Display the hotel_df dataframe
hotel_df

,Hotel Name,Latitude,Longitude,Address,City,Country
0,Riverhouse on the Deschutes,44.0582,-121.3153,"3075 N Hwy 97, Bend",Bend,US
1,Nobu Hotel Riyadh,24.6877,46.7219,"King Fahd Rd, Al Anoud Tower 2, Riyadh",Riyadh,SA
2,Alnuzl Special Hotel,29.9697,40.2064,"3593 طريق الملك عبدالعزيز،, 7521, Sakaka",Sakakah,SA
3,The Lallgarh Palace - A Heritage Hotel,28.0167,73.3000,Lallgarh Palace Complex Opposite Roadways Bus ...,Bikaner,IN
4,Sofitel Legend Old Cataract Aswan,24.0934,32.9070,"Abtal Al Tahrir, Street, Aswan First",Aswān,EG


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [54]:
# Add marker layer ontop of heat map
fig2 = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))